In [22]:
# External packages
import sys
import warnings
warnings.filterwarnings("ignore") # suppress warnings
import awkward as ak
import pandas as pd
# mu2e-cosmic classes
sys.path.extend(["../../src/core", "../../src/utils"])
from io_manager import Load

In [17]:
ana_results = Load(in_path="../../output/results/dev_signal_mix2BB_onspill-LH_aw_noCRV/").load_pkl()
ml_prep_results = Load(in_path="../../output/ml/veto/d/data/CE_mix_onspill-LH_aw/").load_pkl()

[Load] ✅ Initialised with out_path=../../output/results/dev_signal_mix2BB_onspill-LH_aw_noCRV/
[Load] ✅ Successfully loaded results from ../../output/results/dev_signal_mix2BB_onspill-LH_aw_noCRV/results.pkl
[Load] ✅ Initialised with out_path=../../output/ml/veto/d/data/CE_mix_onspill-LH_aw/
[Load] ✅ Successfully loaded results from ../../output/ml/veto/d/data/CE_mix_onspill-LH_aw/results.pkl


In [23]:

# Analysis results
events = ana_results["events"]
dT = events["dev"]["cent_dT"]
mom_cut = ak.any(events["dev"]["within_wide_win"] & events["dev"]["is_reco_electron"], axis=-1)
has_valid_dT = ak.any(dT, axis=-1)
n_in_mom_with_dT_ana = ak.sum(mom_cut & has_valid_dT)
n_in_mom_no_dT_ana = ak.sum(mom_cut & ~has_valid_dT)
n_in_mom_total_ana = ak.sum(mom_cut)

# ML prep results
ml_events = ml_prep_results["events"]
ml_dT = ml_events["dT"]
ml_mom_mag = ml_events["mom_mag"]
ml_mom_cut = (ml_mom_mag > 85) & (ml_mom_mag < 200)
n_in_mom_with_dT_ml = ak.sum(ml_mom_cut & ml_has_valid_dT)
n_in_mom_no_dT_ml = ak.sum(ml_mom_cut & ~ml_has_valid_dT)
n_in_mom_total_ml = ak.sum(ml_mom_cut)

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Metric': [
        'Total events in momentum window',
        'Events with valid dT',
        'Events without valid dT',
        'Fraction with valid dT'
    ],
    'Ana': [
        n_in_mom_total_ana,
        n_in_mom_with_dT_ana,
        n_in_mom_no_dT_ana,
        f"{n_in_mom_with_dT_ana/n_in_mom_total_ana:.2%}"
    ],
    'ML prep': [
        n_in_mom_total_ml,
        n_in_mom_with_dT_ml,
        n_in_mom_no_dT_ml,
        f"{n_in_mom_with_dT_ml/n_in_mom_total_ml:.2%}"
    ],
})

display(comparison_df)

,Metric,Ana,ML prep
0,Total events in momentum window,578372,428440
1,Events with valid dT,428440,428440
2,Events without valid dT,149932,0
3,Fraction with valid dT,74.08%,100.00%


In [7]:
display(results["cut_flow"])

,Cut,Group,Events Passing,Absolute [%],Relative [%],Description
0,No cuts,NaN,1339362,100,100,No selection applied
1,has_trk_front,Preselect,1336979,99.822,99.822,Tracks intersect tracker entrance
2,has_trk_mid,Preselect,1336978,99.822,100,Tracks intersect tracker middle
3,is_reco_electron,Preselect,1327517,99.116,99.292,Electron track fits
4,one_reco_electron,Preselect,1097128,81.914,82.645,One reco electron / event
5,is_downstream,Preselect,1096128,81.840,99.909,Has downstream track (p_z > 0 in tracker)
6,is_truth_electron,Preselect,1096059,81.834,99.994,Track parents are electrons (truth PID)
7,good_trkqual,Tracker,883133,65.937,80.573,Track fit quality > 0.2
8,within_t0,Tracker,655446,48.937,74.218,t0 at tracker mid (640 < t_0 < 1650 ns)
9,within_t0err,Tracker,636329,47.510,97.083,Track fit t0 uncertainty (t0err < 0.9 ns)
